In [1]:
import pymortar
import glob
URL = "http://mortar2_mortar-server_1:5001"
# connect client to Mortar frontend server
c = pymortar.Client(URL)

## Loading Triples/Metadata

Only supports loading from Turtle files for now

In [2]:
# all data/metadata in mortar is grouped by a site name
sitename = "bldg1"

# load in Brick model into the bldg1 'site'
c.load_triple_file(sitename, "sample-data/bldg1/bldg1.ttl")

# load in Brick ontology definitions into the same graph
c.load_triple_file(sitename, "Brick.ttl")

INFO:root:Uploading sample-data/bldg1/bldg1.ttl to http://mortar2_mortar-server_1:5001/insert_triple_file
INFO:root:Uploading Brick.ttl to http://mortar2_mortar-server_1:5001/insert_triple_file


## Bulk Loading

CSV schema is as follows:

- `time`: RFC3339-encoded timestamp
- `value`: value of the stream at the given time
- `id`: the Brick URI of the data stream
- `type`: the Brick type of the data stream
- `label`: the name of the data stream (a label or 'name', not the Brick URI)
- `site`: the name of the graph under which the data should be organized

```
time,id,value,type,label,site
2015-01-01T00:00:00Z,http:/buildsys.org/ontologies/BLDG1#BLDG1.AHU.AHU01A.CCV,0.0,https:/brickschema.org/schema/Brick#Cooling_Valve_Command,BLDG1.AHU.AHU01A.CCV,bldg1
```

In [3]:
# load in CSV data; each row already knows it is for the 'bldg1' site
# 1 CSV file per sensor, 2 days of data
for datafile in glob.glob("sample-data/bldg1/*.csv"):
    c.load_csv(datafile)

INFO:root:Uploading sample-data/bldg1/outhttp___buildsys.org_ontologies_BLDG1#BLDG1.ZONE.AHU04A.RM050_LAB.Zone_Supply_Air_Temp.csv to http://mortar2_mortar-server_1:5001/insert_streaming
INFO:root:Uploading sample-data/bldg1/outhttp___buildsys.org_ontologies_BLDG1#BLDG1.ZONE.AHU04A.RM00D_LAB.Zone_Supply_Air_Temp.csv to http://mortar2_mortar-server_1:5001/insert_streaming
INFO:root:Uploading sample-data/bldg1/outhttp___buildsys.org_ontologies_BLDG1#BLDG1.ZONE.AHU04A.RM243C_LAB.Zone_Air_Temp.csv to http://mortar2_mortar-server_1:5001/insert_streaming
INFO:root:Uploading sample-data/bldg1/outhttp___buildsys.org_ontologies_BLDG1#BLDG1.AHU.AHU03B.Supply_Air_Temp_Setpoint.csv to http://mortar2_mortar-server_1:5001/insert_streaming
INFO:root:Uploading sample-data/bldg1/outhttp___buildsys.org_ontologies_BLDG1#BLDG1.ZONE.AHU07B.RM096_LAB.Zone_Reheat_Valve_Command.csv to http://mortar2_mortar-server_1:5001/insert_streaming
INFO:root:Uploading sample-data/bldg1/outhttp___buildsys.org_ontologies_B

## Streaming / Incremental Updates

In [4]:
import requests

# register a sensor stream with relevant metadata
stream_description = {
    "SourceName": "bldg2",
    "Units": "degF",
    "Name": "mysensortest",
    "BrickURI": "http://example.com#mysensortest", # optional
    "BrickClass": "https://brickschema.org/schema/Brick#Air_Temperature_Sensor", # optional
}
requests.post(f"{URL}/register_stream", json=stream_description)

<Response [200]>

In [5]:
# will register the stream if it doesn't already exist
data = {
    "SourceName": "bldg2",
    "Name": "mysensortest",
    "Readings": [
        ["2015-01-01T00:00:00Z", 72.0],
        ["2015-01-01T00:10:00Z", 72.3],
    ]
}
requests.post(f"{URL}/insert_bulk", json=data)

<Response [200]>

## Querying for Sensors

In [4]:
c.qualify({"temp": "SELECT * WHERE { ?x a brick:Temperature_Sensor }"})
# 'bldg2' has no results because the Brick definition hasn't been loaded in. We'd have to run the following:
# c.qualify({"temp": "SELECT * WHERE { ?x a brick:Temperature_Sensor }"})

       temp
bldg1   833
bldg2     0

In [5]:
c.load_triple_file('bldg2', "Brick.ttl")
# wait a bit for the reasoning to occur, then we can query for Temperature Sensors

INFO:root:Uploading Brick.ttl to http://mortar2_mortar-server_1:5001/insert_triple_file


       temp
bldg1   833
bldg2     0

In [6]:
c.qualify({"temp": "SELECT * WHERE { ?x a brick:Temperature_Sensor }"})

       temp
bldg1   833
bldg2     1